In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Herramientas de Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Modelos (empezaremos con uno simple y uno más complejo)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# Librerías de Explicabilidad (XAI)
import shap
from sklearn.inspection import PartialDependenceDisplay

# Configuración para que las visualizaciones de SHAP se muestren en el notebook
shap.initjs()

In [ ]:
# Carga los datos desde el archivo CSV
file_path = 'dataset.csv'
df = pd.read_csv(file_path)

# Renombrar la columna target
if '`' in df.columns:
    df.rename(columns={'`': 'target_variable'}, inplace=True)

# Definimos las columnas a eliminar
features_to_remove = []

# Definimos las columnas que SÍ usaremos
# (todas las columnas menos 'id', 'target_variable' Y las que queremos eliminar)
features_finales = [
    col for col in df.columns 
    if col not in ['id', 'target_variable'] + features_to_remove
]

# Definimos el target
target = 'target_variable'

print(f"Features a eliminar (ruido): {features_to_remove}")
print(f"Número de features finales: {len(features_finales)}")
print("Features que se usarán para entrenar:")
print(features_finales)

In [ ]:
# 'X_final' solo contiene las columnas que hemos seleccionado
X_final = df[features_finales]

# 'y_final' es nuestro target
y_final = df[target]

print(f"Forma de X_final: {X_final.shape}")
print(f"Forma de y_final: {y_final.shape}")

In [ ]:
# Dividir los datos: 80% para entrenamiento, 20% para prueba
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_final, y_final, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_final
)

print(f"Forma de X_train_final: {X_train_final.shape}")
print(f"Forma de X_test_final: {X_test_final.shape}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier # Importar el nuevo modelo

# 1. Inicializar el modelo Random Forest
# random_state=42 para resultados consistentes
rf_model = RandomForestClassifier(random_state=42)

# 2. Entrenar el modelo con los mismos datos de entrenamiento
rf_model.fit(X_train_final, y_train_final)

# 3. Hacer predicciones en el conjunto de test
y_pred_rf = rf_model.predict(X_test_final)

# 4. Calcular el F1-Score
f1_rf = f1_score(y_test_final, y_pred_rf)

print(f"--- Resultados del Modelo (Random Forest) ---")
print(f"F1-Score: {f1_rf:.4f}")

# 5. Ver el reporte completo
print("\nReporte de Clasificación (Random Forest):")
print(classification_report(y_test_final, y_pred_rf))

In [ ]:
# 1. Importancia de Features (Incorporada en el modelo)
print("--- Insights Globales (Model-Based) ---")
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': features_finales,  # Usar la lista de features finales
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importance_df.head(10))

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df.head(10), x='importance', y='feature')
plt.title('Top 10 Features (Importancia de Gradient Boosting)')
plt.show()


# ---
# 2. Explicabilidad con SHAP (Más Robusto)
print("\n--- Insights Globales (SHAP) ---")

# Calculamos los valores SHAP
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test_final)

# --- SHAP Summary Plot (Beeswarm) ---
print("Generando SHAP Summary Plot (Beeswarm)...")

# Este 'if' comprueba el tipo de shap_values y ejecuta el código correcto
if isinstance(shap_values, list):
    print("Diagnóstico: shap_values es una LISTA. Trazando para la Clase 1 ('Won').")
    shap.summary_plot(
        shap_values[1],  # Usamos el índice [1] para la clase "Won"
        X_test_final, 
        feature_names=features_finales,
        show=False
    )
else:
    print("Diagnóstico: shap_values es una MATRIZ ÚNICA. Trazando directamente.")
    shap.summary_plot(
        shap_values,  # Pasamos la matriz entera directamente
        X_test_final, 
        feature_names=features_finales,
        show=False
    )
        
plt.title("SHAP Summary Plot (Impacto en la predicción)")
plt.show()

# --- SHAP Bar Plot (Importancia media) ---
print("\nGenerando SHAP Bar Plot (Importancia media)...")

# Repetimos la misma lógica para el gráfico de barras
if isinstance(shap_values, list):
    print("Diagnóstico: shap_values es una LISTA. Trazando para la Clase 1 ('Won').")
    shap.summary_plot(
        shap_values[1], # Usamos el índice [1]
        X_test_final, 
        feature_names=features_finales, 
        plot_type="bar",
        show=False
    )
else:
    print("Diagnóstico: shap_values es una MATRIZ ÚNICA. Trazando directamente.")
    shap.summary_plot(
        shap_values, # Pasamos la matriz entera directamente
        X_test_final, 
        feature_names=features_finales, 
        plot_type="bar",
        show=False
    )
        
plt.title("Importancia Media de Features (SHAP)")
plt.show()

In [ ]:
print("--- Insights de Features (Partial Dependence Plots) ---")

# Identifica las features más importantes del Bloque 5
features_to_plot = feature_importance_df['feature'].head(3).tolist()

# Asegurarse de que tenemos features para graficar
if not features_to_plot:
    print("No se identificaron features importantes. Saltando PDP.")
else:
    print(f"Generando PDP para: {features_to_plot}")
    
    # Crear los gráficos
    fig, ax = plt.subplots(figsize=(15, 5), ncols=len(features_to_plot))
    
    # Si solo es una feature, ax no es un array, así que lo ajustamos
    if len(features_to_plot) == 1:
        ax = [ax]
        
    display = PartialDependenceDisplay.from_estimator(
        rf_model,
        X_train_final,
        features_to_plot,
        ax=ax,
        grid_resolution=20 # Número de puntos a calcular en el gráfico
    )
    
    fig.suptitle('Partial Dependence Plots (Efecto promedio de la feature en la predicción)')
    plt.tight_layout()
    plt.show()